<a href="https://colab.research.google.com/github/sanaaria/Master-thesis/blob/main/Run_KG_GCN_DTIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [ ]:
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

In [ ]:
pip install ampligraph

In [ ]:
import ampligraph as ampligraph

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import pandas as pd
import ampligraph as ampligraph
from ampligraph.datasets import load_from_csv

In [ ]:
# from ampligraph.evaluation import train_test_split_no_unseen,generate_corruptions_for_fit
# # from ampligraph.evaluation import train_test_split_no_unseen
from ampligraph.datasets import load_from_csv


In [ ]:
# from ampligraph.evaluation import evaluate_performance
# As of version 1.1.1, Ampligraph removed the 'evaluate_performance' function and instead introduced the 'mrr_score', 'hits_at_k', and 'mean_rank' functions to calculate metrics for evaluating model performance.
# If you are using version 2.0.1, you should be able to use the 'mrr_score', 'hits_at_k', and 'mean_rank' functions to calculate the desired metrics. Here's an example of how you can do this
from ampligraph.evaluation import mrr_score

In [ ]:
from ampligraph.evaluation import mrr_score, hits_at_n_score ,mr_score

In [ ]:
# from ampligraph.evaluation.common import generate_corruptions
from ampligraph.latent_features.layers.corruption_generation import CorruptionGenerationLayerTrain

In [ ]:
from ampligraph.latent_features.layers.scoring import ComplEx

In [ ]:
from ampligraph.latent_features.layers.scoring import TransE

In [ ]:
from ampligraph.latent_features.layers.scoring import DistMult

In [ ]:
# from ampligraph.utils import save_model,restore_model
from ampligraph.utils import save_model
from ampligraph.utils import restore_model

In [ ]:
from tensorflow.python.keras.models import Model

In [ ]:
from tensorflow.python.keras.layers import Dense, Dropout

In [ ]:
from keras.layers import LSTM, Lambda, Layer, Dropout

In [ ]:
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam,Adagrad,Adamax

In [ ]:
from tensorflow import keras

In [ ]:
#load data
################################################################

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#data example: yamanishi_08
dt_08 = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/dt_all_08.txt',delimiter='\t',header=None)
# the script reads a csv file using pandas' read_csv function. This function reads the file from the specified path, which in this case is
# /content/drive/MyDrive/data/yamanishi_08/dt_all_08.txt.

dt_08.columns = ['head','relation','tail']
# the columns of the DataFrame dt_08 are set using the columns attribute. The column names are 'head', 'relation', and 'tail'.

In [ ]:
#kg
# ##This code is written in Python using the pandas library.
# #The goal of this code is to load two text files,
# which contain Knowledge Graph (KG) data, and concatenate them into a single pandas DataFrame.
# The KG data in these text files consists of triples (head, relation, tail), which are essentially edges in a graph.
# The 'head' is the subject, the 'relation' is the predicate, and the 'tail' is the object.

kg1 = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/kg_data/kegg_kg.txt',delimiter='\t',header=None)
# The pd.read_csv() function reads the specified file and creates a DataFrame. The delimiter='\t' argument tells pandas to use tabs as separators.
# The header=None argument tells pandas that the first row of the file does not contain column names.

kg2 = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/kg_data/yamanishi_uniprot_kg.txt',delimiter='\t',header=None)
#This code is similar to the previous one.

kg = pd.concat([kg1,kg2])
#Concatenate the two DataFrames.
#The pd.concat() function concatenates the input DataFrames into a single DataFrame.

kg.index = range(len(kg))
#Reset the index of the concatenated DataFrame.
#The index attribute of a DataFrame represents the index of the rows.
#This line of code resets the index of the concatenated DataFrame so that it starts from 0 and increments by 1.

kg.columns = ['head','relation','tail']
#Set the column names of the concatenated DataFrame.
#This line of code assigns new column names to the concatenated DataFrame.


#The resulting kg DataFrame contains the combined KG data from both text files.
# The DataFrame has three columns: 'head', 'relation', and 'tail'. The rows represent the triples (head, relation, tail) in the KG.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# # نمودار توزیع برای نوع داده‌ها
# sns.countplot(data=kg, x='relation')
# plt.xticks(rotation=90)
# plt.show()


In [ ]:
pip install networkx matplotlib


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt


In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler


# اولین دو خط، دو شیء LabelEncoder را ایجاد می‌کند.
# این اشیاء برای تبدیل متغیرهای دسته‌ای به یک فرمت عددی که برای الگوریتم‌های یادگیری ماشین قابل فهم باشد، استفاده می‌شوند.
# تابع LabelEncoder() دو بار فراخوانی می‌شود تا دو شیء head_le و tail_le ایجاد شوند.
head_le = LabelEncoder()
tail_le = LabelEncoder()
relation_le=LabelEncoder()
# متد fit() بر روی هر دو شیء فراخوانی می‌شود. این متد پارامترهای لازم برای انجام رمزگذاری را محاسبه می‌کند.
head_le.fit(dt_08['head'].values)
tail_le.fit(dt_08['tail'].values)
relation_le.fit(dt_08['relation'].values)
# MinMaxScaler از ماژول preprocessing کتابخانه sklearn وارد می‌شود. این برای مقیاس‌بندی داده‌ها استفاده می‌شود.
mms = MinMaxScaler(feature_range=(0, 1))


In [ ]:
dt_08['head'] = head_le.transform(dt_08['head'].values)
dt_08['tail'] = tail_le.transform(dt_08['tail'].values)
dt_08['relation']=relation_le.transform(dt_08['relation'].values)
print("نتیجه‌ی Label Encoding:")
print(dt_08)

نتیجه‌ی Label Encoding:
      head  relation  tail
0        0         0     0
1      181         0     0
2       11         0     1
3       60         0     1
4        5         0     3
...    ...       ...   ...
5122    55         0   943
5123    79         0   943
5124   335         0   943
5125   210         0   986
5126    63         0   987

[5127 rows x 3 columns]


In [ ]:
dt_08 = mms.fit_transform(dt_08)
print("\nنتیجه‌ی مقیاس‌بندی با MinMaxScaler:")
print(dt_08)


نتیجه‌ی مقیاس‌بندی با MinMaxScaler:
[[0.         0.         0.        ]
 [0.22911392 0.         0.        ]
 [0.01392405 0.         0.00101215]
 ...
 [0.42405063 0.         0.95445344]
 [0.26582278 0.         0.99797571]
 [0.07974684 0.         0.99898785]]


In [ ]:
#فراخوانی شناسه داروها (Drug IDs):
fp_id = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/791drug_struc.csv')['drug_id']

# فراخوانی شناسه پروتئین‌ها و توالی‌های آنها:
df_proseq = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/989proseq.csv')
df_proseq.columns = ['pro_id','pro_ids','seq']

# استخراج شناسه پروتئین‌ها:
pro_id = df_proseq['pro_id']

#فراخوانی ویژگی‌های داروها:
drug_feats = np.loadtxt('/content/drive/MyDrive/data/yamanishi_08/morganfp.txt',delimiter=',')

#فراخوانی ویژگی‌های پروتئین‌ها:
pro_feats = np.loadtxt('/content/drive/MyDrive/data/yamanishi_08/pro_ctd.txt',delimiter=',')

#مقیاس‌بندی ویژگی‌های پروتئین‌ها:
pro_feats_scaled = mms.fit_transform(pro_feats)

# کاهش ابعاد ویژگی‌های پروتئین با استفاده از PCA:
pro_feats_scaled2 = PCA(n_components=100).fit_transform(pro_feats_scaled)

#دوباره مقیاس‌بندی ویژگی‌های کاهش‌یافته:
pro_feats_scaled3 = mms.fit_transform(pro_feats_scaled2)

# ترکیب شناسه‌های داروها با ویژگی‌های آنها:
fp_df = pd.concat([fp_id,pd.DataFrame(drug_feats)],axis=1)

#ترکیب شناسه‌های پروتئین‌ها با ویژگی‌های آنها:
prodes_df = pd.concat([pro_id,pd.DataFrame(pro_feats_scaled3)],axis=1)


In [ ]:
data_path = '/content/drive/MyDrive/data/yamanishi_08/data_folds/warm_start_1_10'


def load_data(i):
    # Read the train_fold csv file. The label is included.
    train = pd.read_csv(data_path+'/train_fold_'+str(i+1)+'.csv')[['head','relation','tail','label']]

    # Select only the positive examples (label == 1) from the train set.
    train_pos = train[train['label']==1]

    # Read the test_fold csv file. The label is included.
    test = pd.read_csv(data_path+'/test_fold_'+str(i+1)+'.csv')[['head','relation','tail','label']]

    # Merge the positive train examples and the knowledge graph into a single dataframe.
    data = pd.concat([train_pos,kg])[['head','relation','tail']]


    # Return the train, train_pos, test, and data dataframes.
    return train,train_pos,test,data



In [ ]:
def roc_auc(y,pred):

  # این خط تابع roc_curve را از ماژول متریک (بخشی از کتابخانه Scikit-Learn) با پارامترهای y و pred فراخوانی می کند.
 #تابع منحنی_roc منحنی ROC را برای مسئله طبقه بندی باینری داده شده، نرخ مثبت کاذب (FPR)، نرخ مثبت واقعی (TPR) و آستانه ها را محاسبه می کند.
    fpr, tpr, thresholds = metrics.roc_curve(y, pred)

    #roc_auc = metrics.auc(fpr, tpr): این خط تابع auc را از ماژول متریک با پارامترهای fpr و tpr فراخوانی می کند.
 # تابع auc مساحت زیر منحنی ROC را محاسبه می‌کند که امتیاز AUC-ROC است.
    roc_auc = metrics.auc(fpr, tpr)

    #return roc_auc: این خط امتیاز AUC-ROC محاسبه شده را برمی گرداند.
    return roc_auc


In [ ]:
from sklearn import metrics

def aupr(y, pred):
    # این خط تابع precision_recall_curve را از ماژول متریک (بخشی از کتابخانه Scikit-Learn) با پارامترهای y و pred فراخوانی می کند.
    # این تابع منحنی Precision-Recall را برای مسئله طبقه بندی باینری محاسبه می کند و دقت، بازیابی و آستانه ها را برمی گرداند.
    precision, recall, thresholds = metrics.precision_recall_curve(y, pred)

    # این خط تابع auc را از ماژول متریک با پارامترهای recall و precision فراخوانی می کند.
    # تابع auc مساحت زیر منحنی Precision-Recall را محاسبه می کند که امتیاز AUPR است.
    aupr_value = metrics.auc(recall, precision)

    # این خط امتیاز AUPR محاسبه شده را برمی گرداند.
    return aupr_value

In [ ]:
from sklearn import metrics

def f1_score_custom(y, pred):
    # این خط تابع f1_score را از ماژول متریک (بخشی از کتابخانه Scikit-Learn) با پارامترهای y و pred فراخوانی می کند.
    # این تابع F1 Score را برای مسئله طبقه بندی باینری محاسبه می کند.
    f1 = metrics.f1_score(y, pred)

    # این خط امتیاز F1 محاسبه شده را برمی گرداند.
    return f1

In [ ]:
from sklearn import metrics

def precision_custom(y, pred):
    # این خط تابع precision_score را از ماژول متریک (بخشی از کتابخانه Scikit-Learn) با پارامترهای y و pred فراخوانی می کند.
    # این تابع دقت را برای مسئله طبقه بندی باینری محاسبه می کند.
    precision = metrics.precision_score(y, pred)

    # این خط امتیاز دقت محاسبه شده را برمی گرداند.
    return precision

In [ ]:
from sklearn import metrics

def accuracy_custom(y, pred):
    # این خط تابع accuracy_score را از ماژول متریک (بخشی از کتابخانه Scikit-Learn) با پارامترهای y و pred فراخوانی می کند.
    # این تابع دقت را برای مسئله طبقه بندی باینری محاسبه می کند.
    accuracy = metrics.accuracy_score(y, pred)

    # این خط امتیاز دقت محاسبه شده را برمی گرداند.
    return accuracy

In [ ]:
from sklearn import metrics

def mcc_custom(y, pred):
    # این خط تابع matthews_corrcoef را از ماژول متریک (بخشی از کتابخانه Scikit-Learn) با پارامترهای y و pred فراخوانی می کند.
    # این تابع MCC را برای مسئله طبقه بندی باینری محاسبه می کند.
    mcc = metrics.matthews_corrcoef(y, pred)

    # این خط امتیاز MCC محاسبه شده را برمی گرداند.
    return mcc

In [ ]:
from sklearn import metrics

def sensitivity_custom(y, pred):
    # این خط تابع recall_score را از ماژول متریک (بخشی از کتابخانه Scikit-Learn) با پارامترهای y و pred فراخوانی می کند.
    # این تابع حساسیت (Sensitivity) را برای مسئله طبقه بندی باینری محاسبه می کند.
    sensitivity = metrics.recall_score(y, pred)

    # این خط امتیاز حساسیت محاسبه شده را برمی گرداند.
    return sensitivity


In [ ]:
from sklearn import metrics

def specificity_custom(y, pred):
    # این خط ماتریس درهم‌ریختگی (Confusion Matrix) را با استفاده از y و pred محاسبه می‌کند.
    cm = metrics.confusion_matrix(y, pred)

    # این خط عناصر ماتریس درهم‌ریختگی را به ترتیب برای TN، FP، FN و TP استخراج می‌کند.
    tn, fp, fn, tp = cm.ravel()

    # این خط ویژگی (Specificity) را با استفاده از TN و FP محاسبه می‌کند.
    specificity = tn / (tn + fp)

    # این خط امتیاز ویژگی محاسبه شده را برمی گرداند.
    return specificity


In [ ]:
def get_gcn_embeddings(model, train_triples, test_triples):
    # استخراج تعبیه‌های head از مدل برای داده‌های آموزشی و آزمایشی
    train_sub_embeddings = model.get_embeddings(train_triples['head'].values, embedding_type='e')
    test_sub_embeddings = model.get_embeddings(test_triples['head'].values, embedding_type='e')

    # استخراج تعبیه‌های tail از مدل برای داده‌های آموزشی و آزمایشی
    train_obj_embeddings = model.get_embeddings(train_triples['tail'].values, embedding_type='e')
    test_obj_embeddings = model.get_embeddings(test_triples['tail'].values, embedding_type='e')

    # ترکیب تعبیه‌های head و tail برای داده‌های آموزشی
    train_feats = np.concatenate([train_sub_embeddings, train_obj_embeddings], axis=1)

    # ترکیب تعبیه‌های head و tail برای داده‌های آزمایشی
    test_feats = np.concatenate([test_sub_embeddings, test_obj_embeddings], axis=1)

    # بازگرداندن تعبیه‌های آماده برای استفاده در GCN
    return train_feats, test_feats


In [ ]:

#به طور خلاصه، تابع get_features یک داده فریم داده، دو فریم داده fp_df و prodes_df و یک متغیر بولی use_pro می گیرد.
#ویژگی‌های دارویی را از fp_df و ویژگی‌های پروتئینی را از prodes_df با استفاده از ستون‌های سر و دم داده‌ها استخراج می‌کند.
#سپس، ویژگی ها را به صورت افقی بر اساس ارزش use_pro به هم متصل می کند. ماتریس ویژگی حاصل به عنوان خروجی تابع برگردانده می شود.
#این خط تابعی به نام get_features را تعریف می کند. به چهار پارامتر داده، fp_df، prodes_df و use_pro نیاز دارد.
def get_features(data,fp_df,prodes_df,use_pro):

    #این خط یک اتصال سمت چپ داده ها و fp_df را در ستون 'head' داده ها و ستون 'drug_id' در fp_df انجام می دهد.
 #پیوستن سمت چپ به این دلیل انجام می‌شود که می‌خواهیم همه رکوردها را از جدول سمت چپ (یعنی داده‌ها) و رکوردهای مطابقت‌شده را از جدول سمت راست (یعنی fp_df) نگه داریم.
 #سپس، کد 1025 ستون (از ستون 5 تا 1029) از دیتافریم حاصل را انتخاب می کند و با استفاده از ویژگی values ​​آن را به یک آرایه numpy تبدیل می کند.
 #نتیجه در متغیر_ویژگی های دارو ذخیره می شود.
    drug_features = pd.merge(data,fp_df,how='left',left_on='head',right_on='drug_id').iloc[:,4:1029].values

   #این خط عملیات مشابه قبلی را انجام می دهد، اما این بار داده ها را به هم می پیوندد و prodes_df را در ستون 'tail' داده و ستون 'pro_id' prodes_df را می پیوندد.
 #دوباره، کد 101 ستون (از ستون 5 تا 105) از دیتافریم حاصل را انتخاب می کند و با استفاده از ویژگی values ​​آن را به یک آرایه numpy تبدیل می کند.
 #نتیجه در متغیر pro_features ذخیره می شود.
    pro_features = pd.merge(data,prodes_df,how='left',left_on='tail',right_on='pro_id').iloc[:,4:105].values

   #این خط مقدار متغیر use_pro را بررسی می کند. اگر True باشد، با استفاده از تابع np.concatenate، drug_features و pro_features را به صورت افقی به هم متصل می کند.
 #اگر use_pro False باشد، مستقیماً ویژگی‌های دارو را به ویژگی متغیر اختصاص می‌دهد.
    if use_pro:
        feature = np.concatenate([drug_features,pro_features],axis=1)
    else:
        feature = drug_features

    #این خط ماتریس ویژگی نهایی را برمی گرداند.
    return feature


In [ ]:
# import numpy as np
# import networkx as nx
# from sklearn.preprocessing import MinMaxScaler

# def get_gcn_input(re_train_all, re_test_all, train_feats, test_feats, train_des, test_des, edge_list_train, edge_list_test, embedding_dim):
#     # 1. ترکیب ویژگی‌ها و توضیحات:
#     train_all_feats = np.concatenate([train_feats, train_des], axis=1)
#     test_all_feats = np.concatenate([test_feats, test_des], axis=1)

#     # 2. استانداردسازی ویژگی‌ها:
#     mms = MinMaxScaler()
#     train_all_feats_scaled = mms.fit_transform(train_all_feats)
#     test_all_feats_scaled = mms.transform(test_all_feats)

#     # 3. ایجاد ماتریس مجاورت (Adjacency Matrix) برای گراف:
#     # از edge_list_train و edge_list_test برای ایجاد گراف‌ها و ماتریس‌های مجاورت استفاده می‌شود.
#     G_train = nx.from_edgelist(edge_list_train)
#     G_test = nx.from_edgelist(edge_list_test)

#     adj_train = nx.adjacency_matrix(G_train).todense()
#     adj_test = nx.adjacency_matrix(G_test).todense()

#     # 4. ساخت ورودی مدل:
#     train_model_input = {
#         'features': train_all_feats_scaled,  # ویژگی‌های گره‌ها
#         'adjacency': adj_train  # ماتریس مجاورت
#     }

#     test_model_input = {
#         'features': test_all_feats_scaled,  # ویژگی‌های گره‌ها
#         'adjacency': adj_test  # ماتریس مجاورت
#     }

#     # 5. بازگرداندن ورودی‌های آماده:
#     return train_model_input, test_model_input


In [ ]:
import numpy as np
import networkx as nx
from sklearn.preprocessing import MinMaxScaler
import torch

def normalize_adj(adj):
    adj = adj + np.eye(adj.shape[0])  # افزودن خودپیوستگی برای ماتریس
    rowsum = np.array(adj.sum(1))
    degree_mat_inv_sqrt = np.power(rowsum, -0.5).flatten()
    degree_mat_inv_sqrt[np.isinf(degree_mat_inv_sqrt)] = 0.
    degree_mat_inv_sqrt = np.diag(degree_mat_inv_sqrt)
    normalized_adj = degree_mat_inv_sqrt.dot(adj).dot(degree_mat_inv_sqrt)
    return normalized_adj


In [ ]:
def to_torch_tensor(data):
    return torch.tensor(data, dtype=torch.float32)

In [ ]:
def get_gcn_input(re_train_all, re_test_all, train_feats, test_feats, train_des, test_des, edge_list_train, edge_list_test, embedding_dim):
    # 1. ترکیب ویژگی‌ها و توضیحات:
    train_all_feats = np.concatenate([train_feats, train_des], axis=1)
    test_all_feats = np.concatenate([test_feats, test_des], axis=1)

    # 2. استانداردسازی ویژگی‌ها:
    mms = MinMaxScaler()
    train_all_feats_scaled = mms.fit_transform(train_all_feats)
    test_all_feats_scaled = mms.transform(test_all_feats)

    # 3. ایجاد ماتریس مجاورت (Adjacency Matrix) برای گراف:
    G_train = nx.from_edgelist(edge_list_train)
    G_test = nx.from_edgelist(edge_list_test)

    adj_train = normalize_adj(nx.adjacency_matrix(G_train).todense())
    adj_test = normalize_adj(nx.adjacency_matrix(G_test).todense())

    # 4. ساخت ورودی مدل:
    train_model_input = {
        'features': to_torch_tensor(train_all_feats_scaled),  # ویژگی‌های گره‌ها
        'adjacency': to_torch_tensor(adj_train)  # ماتریس مجاورت
    }

    test_model_input = {
        'features': to_torch_tensor(test_all_feats_scaled),  # ویژگی‌های گره‌ها
        'adjacency': to_torch_tensor(adj_test)  # ماتریس مجاورت
    }

    # 5. بازگرداندن ورودی‌های آماده:
    return train_model_input, test_model_input


In [ ]:
pip install dgl

In [ ]:
import dgl
import torch
import torch.nn as nn
import torch.optim as optim
from dgl.nn.pytorch import GraphConv
from sklearn.metrics import confusion_matrix, precision_score, f1_score, roc_auc_score, average_precision_score, matthews_corrcoef
import numpy as np

class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.conv = GraphConv(in_feats, out_feats)

    def forward(self, g, features):
        return self.conv(g, features)

class GCNModel(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super(GCNModel, self).__init__()
        self.layer1 = GCNLayer(in_feats, hidden_feats)
        self.layer2 = GCNLayer(hidden_feats, hidden_feats)
        self.fc = nn.Linear(hidden_feats, out_feats)
        self.dropout = nn.Dropout(0.5)

    def forward(self, g, features):
        x = self.layer1(g, features)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.layer2(g, x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = dgl.mean_nodes(g, x)  # میانگین برای ترکیب ویژگی‌ها
        x = self.fc(x)
        return x



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import confusion_matrix, precision_score, f1_score, roc_auc_score, average_precision_score, matthews_corrcoef

def evaluate_metrics(y_true, y_pred):
    # تبدیل پیش‌بینی‌ها به مقادیر باینری
    y_pred_binary = (y_pred > 0.5).astype(int)

    # محاسبه متریک‌ها
    cm = confusion_matrix(y_true, y_pred_binary)
    tn, fp, fn, tp = cm.ravel()

    sen = tp / (tp + fn)  # Sensitivity (Recall)
    spe = tn / (tn + fp)  # Specificity
    acc = (tp + tn) / (tp + tn + fp + fn)  # Accuracy
    precision = precision_score(y_true, y_pred_binary)
    f1 = f1_score(y_true, y_pred_binary)
    auroc = roc_auc_score(y_true, y_pred)
    aupr = average_precision_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred_binary)

    return sen, spe, acc, precision, f1, auroc, aupr, mcc

In [ ]:
def train_gcn_and_evaluate(graphs, features, labels, test_graphs, test_features, test_labels, batch_size=64, epochs=1):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Prepare DGL graphs
    train_graph = graphs[0].to(device)
    test_graph = test_graphs[0].to(device)

    train_features = torch.tensor(features, dtype=torch.float32).to(device)
    test_features = torch.tensor(test_features, dtype=torch.float32).to(device)
    train_labels = torch.tensor(labels, dtype=torch.float32).to(device)
    test_labels = torch.tensor(test_labels, dtype=torch.float32).to(device)

    num_feats = train_features.shape[1]
    num_classes = 1  # برای طبقه‌بندی دودویی

    model = GCNModel(num_feats, 128, num_classes).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    loss_fn = nn.BCEWithLogitsLoss()

    # آموزش مدل
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(train_graph, train_features)
        loss = loss_fn(outputs.squeeze(), train_labels)
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}")

    # پیش‌بینی بر روی داده‌های تست
    model.eval()
    with torch.no_grad():
        # برای داده‌های تست
        test_outputs = model(test_graph, test_features)
        y_pred_test = torch.sigmoid(test_outputs).cpu().numpy().flatten()
        y_true_test = test_labels.cpu().numpy()

        # برای داده‌های آموزش
        train_outputs = model(train_graph, train_features)
        y_pred_train = torch.sigmoid(train_outputs).cpu().numpy().flatten()
        y_true_train = train_labels.cpu().numpy()

    # ارزیابی مدل بر روی داده‌های تست
    sen_test, spe_test, acc_test, precision_test, f1_test, auroc_test, aupr_test, mcc_test = evaluate_metrics(y_true_test, y_pred_test)

    # ارزیابی مدل بر روی داده‌های آموزش
    sen_train, spe_train, acc_train, precision_train, f1_train, auroc_train, aupr_train, mcc_train = evaluate_metrics(y_true_train, y_pred_train)

    # چاپ نتایج تست
    print("\n*** نتایج روی داده‌های تست ***")
    print(f"Sen: {sen_test:.4f}")
    print(f"Spe: {spe_test:.4f}")
    print(f"ACC: {acc_test:.4f}")
    print(f"Precision: {precision_test:.4f}")
    print(f"F1 Score: {f1_test:.4f}")
    print(f"AUROC: {auroc_test:.4f}")
    print(f"AUPR: {aupr_test:.4f}")
    print(f"MCC: {mcc_test:.4f}")

    # چاپ نتایج آموزش
    print("\n*** نتایج روی داده‌های آموزش ***")
    print(f"Sen: {sen_train:.4f}")
    print(f"Spe: {spe_train:.4f}")
    print(f"ACC: {acc_train:.4f}")
    print(f"Precision: {precision_train:.4f}")
    print(f"F1 Score: {f1_train:.4f}")
    print(f"AUROC: {auroc_train:.4f}")
    print(f"AUPR: {aupr_train:.4f}")
    print(f"MCC: {mcc_train:.4f}")

    return model, sen_test, spe_test, acc_test, precision_test, f1_test, auroc_test, aupr_test, mcc_test, y_pred_test

In [ ]:
# def train_gcn_and_evaluate(graphs, features, labels, test_graphs, test_features, test_labels, batch_size=64, epochs=1):
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#     # Prepare DGL graphs
#     train_graph = graphs[0].to(device)
#     test_graph = test_graphs[0].to(device)

#     train_features = torch.tensor(features, dtype=torch.float32).to(device)
#     test_features = torch.tensor(test_features, dtype=torch.float32).to(device)
#     train_labels = torch.tensor(labels, dtype=torch.float32).to(device)
#     test_labels = torch.tensor(test_labels, dtype=torch.float32).to(device)

#     num_feats = train_features.shape[1]
#     num_classes = 1  # برای طبقه‌بندی دودویی

#     model = GCNModel(num_feats, 128, num_classes).to(device)
#     optimizer = optim.Adam(model.parameters(), lr=0.01)
#     loss_fn = nn.BCEWithLogitsLoss()

#     # آموزش مدل
#     for epoch in range(epochs):
#         model.train()
#         optimizer.zero_grad()
#         outputs = model(train_graph, train_features)
#         loss = loss_fn(outputs.squeeze(), train_labels)
#         loss.backward()
#         optimizer.step()
#         print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}")

#     # پیش‌بینی بر روی داده‌های تست
#     model.eval()
#     with torch.no_grad():
#         test_outputs = model(test_graph, test_features)
#         y_pred = torch.sigmoid(test_outputs).cpu().numpy().flatten()
#         y_true = test_labels.cpu().numpy()

#     # ارزیابی مدل
#     sen, spe, acc, precision, f1, auroc, aupr, mcc = evaluate_metrics(y_true, y_pred)

#     # چاپ نتایج
#     print(f"Sen: {sen:.4f}")
#     print(f"Spe: {spe:.4f}")
#     print(f"ACC: {acc:.4f}")
#     print(f"Precision: {precision:.4f}")
#     print(f"F1 Score: {f1:.4f}")
#     print(f"AUROC: {auroc:.4f}")
#     print(f"AUPR: {aupr:.4f}")
#     print(f"MCC: {mcc:.4f}")

#     return model, sen, spe, acc, precision, f1, auroc, aupr, mcc, y_pred

# # مثال استفاده
# # graphs, features, labels باید به درستی تعریف شوند و به تابع ارسال شوند.
# # test_graphs, test_features, test_labels نیز باید به درستی تعریف شوند و به تابع ارسال شوند.
# # model = train_gcn_and_evaluate(graphs, features, labels, test_graphs, test_features, test_labels)


In [ ]:
from ampligraph.latent_features import ScoringBasedEmbeddingModel

In [ ]:
import ampligraph.latent_features.loss_functions as lfs

In [ ]:
from ampligraph.latent_features.loss_functions import get as get_loss

In [ ]:
from ampligraph.latent_features.regularizers import get as get_regularizer

In [ ]:
def knowledge_graph(data):
    k = 400  # embedding dimension
    eta = 10  # number of negative samples per positive sample
    epochs = 1  # number of training epochs
    batches_count = 10000  # number of batches
    optim = tf.keras.optimizers.Adam(learning_rate=1e-3)
    loss = get_loss('pairwise', {'margin': 0.5})
    regularizer = get_regularizer('LP', {'p': 2, 'lambda': 1e-5})

    # Create the DistMult model
    model = ScoringBasedEmbeddingModel(
          k=k,
          eta=eta,
          scoring_type="DistMult",
          # optimizer="Adam",
          # loss="PairwiseMargin",
          # regularizer="LP",
          # regularizer_weight=1e-5,
          seed=42,
      )


    model.compile(optimizer=optim, loss=loss, entity_relation_regularizer=regularizer)

     ###earlystpe alakie
    checkpoint = tf.keras.callbacks.EarlyStopping(
       monitor="val_loss",
       min_delta=0,
       patience=5,
       verbose=1,
       mode='max',
       restore_best_weights=True
)
  ###
    model.fit(data.values,
              batch_size=10000,
              epochs=3 ,                  # Number of training epochs
              # # validation_freq=20,           # Epochs between successive validation
              # validation_burn_in=10,       # Epoch to start validation
              # validation_data=train_pos[['head','relation','tail']].values,   # Validation data
              # validation_filter=dt_08.values,     # Filter positives from validation corruptions
              callbacks=[checkpoint],       # Early stopping callback (more from tf.keras.callbacks are supported)
              # verbose=True                  # Enable stdout messages
              )
    return model

In [ ]:
def train(i,test_num_neg,train_num_neg,embedding_dim,n_components,use_pro,patience):

    #This function loads the train and test data for the current fold.
    train,train_pos,test,data = load_data(i)
    model=knowledge_graph(data)
    columns = ['head','relation','tail']
    re_train_all = train[columns]
    re_test_all = test[columns]
    train_label = train['label']
    test_label = test['label'].values
    test_score = model.predict(test[columns])
    #This function generates the embeddings for the train and test data.
    train_dense_features,test_dense_features = get_scaled_embeddings(model,re_train_all,re_test_all,False,n_components)







    #This function generates the additional features (chemical features, proline features, etc.) for the train and test data.
    train_des = get_features(re_train_all,fp_df,prodes_df,use_pro)
    test_des = get_features(re_test_all,fp_df,prodes_df,use_pro)



    #This function generates the input for the gcn model, which includes the combined features from the embeddings and additional features.
    train_model_input,test_model_input = get_gcn_input(re_train_all,re_test_all,
                                                                    train_dense_features,test_dense_features,
                                                                    train_des,test_des,
                                                                    embedding_dim,n_components)

    train_label = np.array(train_label)
    test_label = np.array(test_label)

    model, sen_test, spe_test, acc_test, precision_test, f1_test, auroc_test, aupr_test, mcc_test, y_pred_test= train_gcn_and_evaluate(train_model_input, train_label, test_model_input, test_label, batch_size=64, epochs=1)


    return   model, sen_test, spe_test, acc_test, precision_test, f1_test, auroc_test, aupr_test, mcc_test, y_pred_test

In [ ]:
Sen = []
Spe = []
ACC = []
Precision = []
F1=[]
AUROC=[]
AUPR=[]
MCC=[]
for i in range(1):

  # print(i) prints the current iteration of the for loop.
    print(i)

    #train() is a function that runs the experiment for a given fold (i), given number of splits (10), given number of recommendations per user (10),
    #given number of features (50), given number of embeddings (200), given whether to use attention or not (True), and given the length of the session sequence (10).
    # It returns several metrics and arrays of user and item embeddings.
    model,sen, spe, acc, precision, f1, auroc, aupr, mcc,re_train_all,train_label,re_test_all,test_label,pred_y = train(i,10,10,50,200,True,10)  #stores the return values of the train() function into variables.
    #assign the ground truth labels (train_label and test_label) to the respective users in the train and test sets.
    re_train_all['label'] = train_label
    re_test_all['label'] = test_label

    #re_test_all['pred'] = pred_y stores the predicted ratings (pred_y) for each user in the test set.
    re_test_all['pred'] = pred_y

    #ROC.append(roc), PR.append(pr), ROC_s.append(roc_s), PR_s.append(pr_s) append the ROC, PR, ROC_s, and PR_s values to their respective lists for each fold.
    Sen.append(sen)
    Spe.append(spe)
    ACC.append(acc)
    Precision.append(precision)
    F1.append(f1)
    AUROC.append(auroc)
    AUPR.append(aupr)
    MCC.append(mcc)

#creates an empty pandas DataFrame.
stable_metrics = pd.DataFrame()

# store the ROC, PR, ROC_s, and PR_s values in the respective columns of the DataFrame.
stable_metrics['sen'] = Sen
stable_metrics['spe'] = Spe
stable_metrics['acc'] = ACC
stable_metrics['precision'] =Precision
stable_metrics['f1'] = F1
stable_metrics['auroc'] = AUROC
stable_metrics['aupr'] = AUPR
stable_metrics['mcc'] = MCC
#prints the summary statistics of the metrics.
stable_metrics.describe()


0
Epoch 1/3
12/12 [==============================] - 1s 116ms/step - loss: 45584.6719
Epoch 2/3
12/12 [==============================] - 0s 36ms/step - loss: 45561.8906
Epoch 3/3
12/12 [==============================] - 0s 39ms/step - loss: 45516.5586
(1924, 1)
